In [1]:
#Este programa contém os códigos para a aula de Cidades Inteligentes

Para a construção de um modelo utilizando o Spark e MLlib são necessários 6 passos:

#1) Construir e iniciar a seção SPARK
#2) Implementar o carregamento dos dados para o spark: Carregar o arquivo, especificar o formato desejado e lê os dados como um Dataframe do Spark
#3) Identificar as características a serem utilizadas para treinamento e teste do modelo
#4) Instanciar as classes e os objetos dos algoritmos a serem utilizados
#5) Utilizar o método fit() para realizar o treinamento do modelo
#5) Avaliar o modelo

In [3]:
%fs ls /FileStore/tables

path,name,size
dbfs:/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,463094961
dbfs:/FileStore/tables/d1995_07_01-24d0c.json,d1995_07_01-24d0c.json,7451741
dbfs:/FileStore/tables/d1995_07_02-c3f44.json,d1995_07_02-c3f44.json,9572086
dbfs:/FileStore/tables/d1995_07_03-f99af.json,d1995_07_03-f99af.json,14125303
dbfs:/FileStore/tables/d1995_07_04-c5a7f.json,d1995_07_04-c5a7f.json,12030333
dbfs:/FileStore/tables/d1995_07_05-0f261.json,d1995_07_05-0f261.json,14662194
dbfs:/FileStore/tables/d1995_07_06-20619.json,d1995_07_06-20619.json,15557682
dbfs:/FileStore/tables/d1995_07_07-2dd8d.json,d1995_07_07-2dd8d.json,15279295
dbfs:/FileStore/tables/d1995_07_08-83302.json,d1995_07_08-83302.json,7033852
dbfs:/FileStore/tables/d1995_07_09-f75d3.json,d1995_07_09-f75d3.json,5589076


In [4]:
#importando as bibliotecas 
from pyspark.ml.classification import LogisticRegression   #utilizada para realizar a classifcação
from pyspark.ml.feature import RegexTokenizer    #utilizada para realizar a tokenização (divisão da senteça em palavras)
from pyspark.ml.feature import StopWordsRemover   #utilizada para remover as stopwords (palavras "sem sentido" para a análise)
from pyspark.ml.feature import CountVectorizer  #transforma as palavras em vetores
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler   #utilizadsa para transformar os dados
from pyspark.ml import Pipeline #utilizada para criar o pipeline de transformações
from pyspark.sql.functions import col #utilizada para selecionar as colunas
from pyspark.sql import SparkSession  #utilizada para iniciar a seção do spark
from pyspark.ml.evaluation import MulticlassClassificationEvaluator  #utilizado para realizar a avaliação de classificadores não binários

In [5]:
spark = SparkSession.builder.appName("PrevisaoDeCrimes").getOrCreate()  #inicia a seção do spark

In [6]:
diretorioDataset="/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv"  #diretório que contém o arquivo a ser utilizado

In [7]:
data = spark.read.format("csv").options(header="true", inferschema="true").load(diretorioDataset)  #realiza a leitura do dataset

In [8]:
data.columns  #apresenta as colunas do dataset

Out[6]: ['IncidntNum',
 'Category',
 'Descript',
 'DayOfWeek',
 'Date',
 'Time',
 'PdDistrict',
 'Resolution',
 'Address',
 'X',
 'Y',
 'Location',
 'PdId']

In [9]:
data.count() #conta a quantidade de registros existente no dataset (linhas) - justifica ser distribuído 

Out[38]: 2215024

In [10]:
data.printSchema()  #realiza o print do schema para o bando de dados lido

root
-- IncidntNum: integer (nullable = true)
-- Category: string (nullable = true)
-- Descript: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- Date: string (nullable = true)
-- Time: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- X: double (nullable = true)
-- Y: double (nullable = true)
-- Location: string (nullable = true)
-- PdId: long (nullable = true)

In [11]:
data.show(10)  #realiza o print do dataset utilizado

+----------+--------------------+--------------------+---------+----------+-----+----------+--------------------+--------------------+-----------------+----------------+--------------------+--------------+
IncidntNum| Category| Descript|DayOfWeek| Date| Time|PdDistrict| Resolution| Address| X| Y| Location| PdId|
+----------+--------------------+--------------------+---------+----------+-----+----------+--------------------+--------------------+-----------------+----------------+--------------------+--------------+
 110308742| MISSING PERSON| FOUND PERSON| Friday|04/15/2011|16:00| TARAVAL| LOCATED|0 Block of CRESPI DR|-122.476327815126|37.7196874952717|POINT (-122.47632...|11030874275000|
 90316914| WARRANTS| WARRANT ARREST| Thursday|03/26/2009|12:28| SOUTHERN| ARREST, BOOKED|1000 Block of MAR...|-122.411339562557|37.7812707434494|POINT (-122.41133...| 9031691463010|
 130132311| SECONDARY CODES| DOMESTIC VIOLENCE| Friday|02/15/2013|02:00| PARK| ARREST, BOOKED|CASTRO ST / MARKE...|-122.435187699349|37.7626702770872|POINT (-122.43518...|13013231115200|
 80052833| OTHER OFFENSES|DRIVERS LICENSE, ...| Tuesday|01/15/2008|20:13| MISSION| ARREST, CITED|VALENCIA ST / 16T...|-122.421886357981|37.7649178909424|POINT (-122.42188...| 8005283365016|
 70535770| SECONDARY CODES| DOMESTIC VIOLENCE| Saturday|05/26/2007|18:10| MISSION| ARREST, BOOKED|2800 Block of FOL...|-122.413935404044|37.7516932889916|POINT (-122.41393...| 7053577015200|
 130407330| OTHER OFFENSES| TRAFFIC VIOLATION| Friday|05/17/2013|21:56| SOUTHERN| ARREST, CITED|500 Block of STEV...|-122.410793915482| 37.780832063096|POINT (-122.41079...|13040733065015|
 136080803| LARCENY/THEFT|GRAND THEFT FROM ...| Sunday|04/07/2013|12:30| SOUTHERN| NONE|800 Block of BRYA...|-122.403404791479| 37.775420706711|POINT (-122.40340...|13608080306244|
 71177357|FORGERY/COUNTERFE...|FORGERY, DRIVERS ...| Friday|11/16/2007|17:00| NORTHERN|PROSECUTED BY OUT...|2500 Block of VAN...|-122.423907410647|37.7990497607213|POINT (-122.42390...| 7117735709120|
 60606109| LARCENY/THEFT|PETTY THEFT FROM ...|Wednesday|06/07/2006|19:45| NORTHERN| NONE|BUSH ST / BUCHANA...|-122.430170528945|37.7874391353148|POINT (-122.43017...| 6060610906243|
 50719825| ASSAULT|BATTERY OF A POLI...| Tuesday|06/28/2005|01:02| BAYVIEW| ARREST, BOOKED|NAPOLEON ST / EVA...|-122.396010154392|37.7472622922111|POINT (-122.39601...| 5071982504154|
+----------+--------------------+--------------------+---------+----------+-----+----------+--------------------+--------------------+-----------------+----------------+--------------------+--------------+
only showing top 10 rows

Vamos eliminar algumas colunas, pois queremos classificar o tipo de crime.

In [13]:
drop_data = ['IncidntNum','Date','Time', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y','PdId', 'Location']  #indica quais as colunas vamos eliminar (vamos ficar apenas com "Category" e "Descript")

In [14]:
data = data.select([column for column in data.columns if column not in drop_data]) #aplica a seleção apenas da colunas que não estão na lista "drop_data"

In [15]:
data.show(10)  #realiza o print do "novo" dataset

+--------------------+--------------------+
 Category| Descript|
+--------------------+--------------------+
 MISSING PERSON| FOUND PERSON|
 WARRANTS| WARRANT ARREST|
 SECONDARY CODES| DOMESTIC VIOLENCE|
 OTHER OFFENSES|DRIVERS LICENSE, ...|
 SECONDARY CODES| DOMESTIC VIOLENCE|
 OTHER OFFENSES| TRAFFIC VIOLATION|
 LARCENY/THEFT|GRAND THEFT FROM ...|
FORGERY/COUNTERFE...|FORGERY, DRIVERS ...|
 LARCENY/THEFT|PETTY THEFT FROM ...|
 ASSAULT|BATTERY OF A POLI...|
+--------------------+--------------------+
only showing top 10 rows

Preparando os dados .... Textual

#1) Tokenização
#2) Remoção de Stopwords
#3) Vetorização (texto -> vetores)

In [18]:
# aplicando expressões regulares (regular expression)
re_Tokenizer = RegexTokenizer(inputCol="Descript",outputCol="words", pattern="\\W")

In [19]:
# remoção de stopwords
stop_words = ["http","https","amp","rt","t","c","the"]   #cria o vetor que contém as palavras que não geram sentido ao texto
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stop_words)  #aplica a remoção das stopwords contidas no vetor

In [20]:
# Transformando as palavras em vetores (BagOfWords)
count_vectors = CountVectorizer(inputCol="filtered",outputCol="features", vocabSize=10000, minDF=5)

In [21]:
# aplicando a transformação (string -> numerica) de cada uma das categorias
label_string_Idx = StringIndexer(inputCol = "Category", outputCol = "label")

In [22]:
#define todas as operação através do pipeline
pipeline = Pipeline(stages=[re_Tokenizer, stop_words_remover,count_vectors, label_string_Idx]) #diz a sequência das transformações a serem realizadas

In [23]:
# Realmente, aplica as transformações 
pipeline_fit = pipeline.fit(data)  #aplica as transformações 
newDataset = pipeline_fit.transform(data)

In [24]:
newDataset.show(10, False)

+----------------------+-------------------------------------+------------------------------------------+------------------------------------------+-----------------------------------------------------+-----+
Category |Descript |words |filtered |features |label|
+----------------------+-------------------------------------+------------------------------------------+------------------------------------------+-----------------------------------------------------+-----+
MISSING PERSON |FOUND PERSON |[found, person] |[found, person] |(886,[32,39],[1.0,1.0]) |10.0 |
WARRANTS |WARRANT ARREST |[warrant, arrest] |[warrant, arrest] |(886,[17,33],[1.0,1.0]) |7.0 |
SECONDARY CODES |DOMESTIC VIOLENCE |[domestic, violence] |[domestic, violence] |(886,[95,96],[1.0,1.0]) |13.0 |
OTHER OFFENSES |DRIVERS LICENSE, SUSPENDED OR REVOKED|[drivers, license, suspended, or, revoked]|[drivers, license, suspended, or, revoked]|(886,[10,23,26,29,30],[1.0,1.0,1.0,1.0,1.0]) |1.0 |
SECONDARY CODES |DOMESTIC VIOLENCE |[domestic, violence] |[domestic, violence] |(886,[95,96],[1.0,1.0]) |13.0 |
OTHER OFFENSES |TRAFFIC VIOLATION |[traffic, violation] |[traffic, violation] |(886,[12,34],[1.0,1.0]) |1.0 |
LARCENY/THEFT |GRAND THEFT FROM LOCKED AUTO |[grand, theft, from, locked, auto] |[grand, theft, from, locked, auto] |(886,[0,2,3,4,5],[1.0,1.0,1.0,1.0,1.0]) |0.0 |
FORGERY/COUNTERFEITING|FORGERY, DRIVERS LICENSE OR ID-CARD |[forgery, drivers, license, or, id, card] |[forgery, drivers, license, or, id, card] |(886,[10,23,26,58,109,159],[1.0,1.0,1.0,1.0,1.0,1.0])|14.0 |
LARCENY/THEFT |PETTY THEFT FROM LOCKED AUTO |[petty, theft, from, locked, auto] |[petty, theft, from, locked, auto] |(886,[0,2,4,5,7],[1.0,1.0,1.0,1.0,1.0]) |0.0 |
ASSAULT |BATTERY OF A POLICE OFFICER |[battery, of, a, police, officer] |[battery, of, a, police, officer] |(886,[1,9,21,107,122],[1.0,1.0,1.0,1.0,1.0]) |3.0 |
+----------------------+-------------------------------------+------------------------------------------+------------------------------------------+-----------------------------------------------------+-----+
only showing top 10 rows

In [25]:
#Dividindo os dados entre treinamento e teste
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100) #divide o datasete entre 70% para treinamento e 30% para teste

In [26]:
print("Comprimento do Treinamento: " + str(trainingData.count())) #conta a quantidade de registros existentes no dataset
print("Comprimento do Teste: " + str(testData.count()))

Comprimento do Treinamento: 1549670
Comprimento do Teste: 665354

In [27]:
#Constrói o modelo de classificação 
regressor = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

In [28]:
# aplica a regressão aos dados de treinamento
modeloRegressaoL = regressor.fit(trainingData)  

In [29]:
#realiza a previsão utilizando o modelo de regressão logística
previsao = modeloRegressaoL.transform(testData)

In [30]:
#avalia o modelo de previsão construído
avaliacao = MulticlassClassificationEvaluator(predictionCol="prediction",metricName='f1') #funcao utilizada para realizar a avaliação de classifcadores com várias classes
avaliacao.evaluate(previsao)  #accuracy, areaUnderROC, accuracy

Out[47]: 0.9724319783834035